In [56]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [67]:
def plot(trace_num, x_data, y_data, xlable = 'xlable', ylable = 'ylable',
                legend = 'legend', title = 'title', mode='lines'):
    fig = go.Figure()
    for i in range(trace_num):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode=mode, name=legend[i], showlegend = True))
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
    fig.show()

In [58]:
data = np.loadtxt('data_group6.txt')

In [71]:
plot(1, [data[:,3]], [data[:,2]], 
           'monthly sunspot number', 'monthly solar radio flux at 10.7 cm',
           ['dataset points'], 'Sunspot number to solar radio flux F10.7cm dependancy',
           mode = 'markers')

In [60]:
sunspot_num_smth = np.empty(data.shape[0])
solar_flux_smth = np.empty(data.shape[0])
sunspot_num_smth[:6] = np.ones([1, 6]) * np.mean(data[:6,3])
sunspot_num_smth[-6:] = np.ones([1, 6]) * np.mean(data[-6:,3])
solar_flux_smth[:6] = np.ones([1, 6]) * np.mean(data[:6,2])
solar_flux_smth[-6:] = np.ones([1, 6]) * np.mean(data[-6:,2])
for i in range(6, data.shape[0] - 6):
    sunspot_num_smth[i] = 1/24 * (data[i-6][3] + data[i+6][3]) + 1/12 * (np.sum(data[i-5:i+6, 3]))
    solar_flux_smth[i] = 1/24 * (data[i-6][2] + data[i+6][2]) + 1/12 * (np.sum(data[i-5:i+6, 2]))

In [75]:
plot(2, [data[:,0], data[:,0]], [sunspot_num_smth, solar_flux_smth],
     legend = ['Sunspot number smooth', 'solar ratio flux smooth'])

In [8]:
F_exp = data[:, 2]
F = np.empty(F_exp.shape)
R = np.ones([data.shape[0], 4])
beta = np.ones(4)
for i in range(4):
    R[:, i] = data[:, 3]**i
beta = np.linalg.inv((R.T.dot(R))).dot(R.T).dot(F_exp)

In [10]:
F = beta[0]*np.ones(data.shape[0])+beta[1]*data[:,3]+beta[2]*data[:,3]**2+beta[3]*data[:,3]**3

In [81]:
plot(2, [data[:,0], data[:,0]], [F, solar_flux_smth],
     legend = ['solar radio flux linear regression', 'solar ratio flux smooth'])